# Homework 7_Session 11. Neural networks

The purpose of the below is to classify days over years 2017-2018 by their corresponding mobility patterns between 10 zones in Taipei (quantified by an aggregated temporal network of subway ridership flows across the city)

In [1]:
#use Python 3.7
import numpy as np
import pandas as pd
import keras
from keras.datasets import mnist
from sklearn.preprocessing import MinMaxScaler
from keras.models import Model, Sequential
from keras.layers import Input, Dense
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
#read the data
workfolder='';
TNet=pd.read_csv(workfolder+'taipeiD_TNet2.csv',header=None);

In [3]:
TNet.head() 
#each row represents a 10x10 adjacency matrix of the normalized Taipei subway mobility network between 10 zones flattened into a 100x1 row corresponding to a single day
#days start at jan-1-2017

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.017943,0.005415,0.003590,0.008316,0.007859,0.012942,0.012196,0.019543,0.001196,0.003327,...,0.002529,0.001533,0.001860,0.002375,0.005408,0.008922,0.003945,0.011075,0.005073,0.012708
1,0.021283,0.005215,0.003530,0.009359,0.007803,0.014288,0.011185,0.019044,0.001382,0.003499,...,0.002803,0.001757,0.001783,0.002549,0.005515,0.009650,0.003596,0.009618,0.005946,0.013709
2,0.028988,0.006511,0.005591,0.012970,0.007816,0.015878,0.010973,0.015768,0.002252,0.005388,...,0.004649,0.002555,0.002672,0.004291,0.007385,0.009558,0.004293,0.008791,0.010040,0.016301
3,0.029534,0.006471,0.005615,0.013017,0.007717,0.016098,0.011182,0.015815,0.002325,0.005443,...,0.004611,0.002473,0.002636,0.004195,0.007255,0.009487,0.004316,0.008729,0.010296,0.016437
4,0.029333,0.006525,0.005727,0.013098,0.007692,0.016358,0.011000,0.015677,0.002344,0.005527,...,0.004694,0.002515,0.002677,0.004222,0.007269,0.009921,0.004387,0.008923,0.010381,0.016914


In [4]:
#convert to an array and scale the data
X=np.array(TNet);

In [5]:
X=MinMaxScaler(feature_range=(0, 1), copy=True).fit_transform(X)

In [6]:
X.shape

(669, 100)

In [7]:
#define day of the week corresponding to each day of observation; 0-Sunday, 1-Monday,...,6-Saturday
y=np.array(range(669))%7; y[:10]

array([0, 1, 2, 3, 4, 5, 6, 0, 1, 2])

In [8]:
yc=np_utils.to_categorical(y) #get categorical binary variables isSunday, isMonday,...
yc[:5]

array([[1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.]], dtype=float32)

In [9]:
X_test=X[400:,:]; X_train=X[:400,:]; #split the data into training and test
y_test=yc[400:,:]; y_train=yc[:400,:]

## Task 1. Classify weekdays/weekends
Label the rows with ones for weekends, zeros for weekdays.
Train a neural network with 4 layers of 30,10,3 and 1 (output) neurons over the training sample against this label, evaluating its performance over the test sample. Report the acheived accuracy (categorical) over the test sample

First three layers use relu activation function, last one - sigmoid.
Use loss='binary_crossentropy', optimizer='adam', 100 epochs, batch_size=20. 

In [10]:
label=[]
for i in y:
    if i==0 or i==6:
        j=1
    else:
        j=0
    label.append(j)

In [11]:
label=np.array(label)

In [12]:
label.shape

(669,)

In [13]:
y_test=label[400:]; y_train=label[:400]

In [14]:
y_test.shape

(269,)

In [15]:
y_train.shape

(400,)

In [16]:
model = Sequential()
model.add(Dense(units=30,activation='relu',input_dim=100))
model.add(Dense(units=10,activation='relu'))
model.add(Dense(units=3,activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['categorical_accuracy'])
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,batch_size=20,verbose=2)
scores = model.evaluate(X_test,y_test,verbose=0)
y_predict=model.predict(X_test)

Train on 400 samples, validate on 269 samples
Epoch 1/100
 - 0s - loss: 0.6497 - categorical_accuracy: 1.0000 - val_loss: 0.6083 - val_categorical_accuracy: 1.0000
Epoch 2/100
 - 0s - loss: 0.5885 - categorical_accuracy: 1.0000 - val_loss: 0.5814 - val_categorical_accuracy: 1.0000
Epoch 3/100
 - 0s - loss: 0.5693 - categorical_accuracy: 1.0000 - val_loss: 0.5658 - val_categorical_accuracy: 1.0000
Epoch 4/100
 - 0s - loss: 0.5569 - categorical_accuracy: 1.0000 - val_loss: 0.5555 - val_categorical_accuracy: 1.0000
Epoch 5/100
 - 0s - loss: 0.5471 - categorical_accuracy: 1.0000 - val_loss: 0.5459 - val_categorical_accuracy: 1.0000
Epoch 6/100
 - 0s - loss: 0.5345 - categorical_accuracy: 1.0000 - val_loss: 0.5362 - val_categorical_accuracy: 1.0000
Epoch 7/100
 - 0s - loss: 0.5248 - categorical_accuracy: 1.0000 - val_loss: 0.5271 - val_categorical_accuracy: 1.0000
Epoch 8/100
 - 0s - loss: 0.5169 - categorical_accuracy: 1.0000 - val_loss: 0.5199 - val_categorical_accuracy: 1.0000
Epoch 9/10

Epoch 70/100
 - 0s - loss: 0.0141 - categorical_accuracy: 1.0000 - val_loss: 0.0785 - val_categorical_accuracy: 1.0000
Epoch 71/100
 - 0s - loss: 0.0143 - categorical_accuracy: 1.0000 - val_loss: 0.0751 - val_categorical_accuracy: 1.0000
Epoch 72/100
 - 0s - loss: 0.0140 - categorical_accuracy: 1.0000 - val_loss: 0.0904 - val_categorical_accuracy: 1.0000
Epoch 73/100
 - 0s - loss: 0.0280 - categorical_accuracy: 1.0000 - val_loss: 0.1005 - val_categorical_accuracy: 1.0000
Epoch 74/100
 - 0s - loss: 0.0192 - categorical_accuracy: 1.0000 - val_loss: 0.0735 - val_categorical_accuracy: 1.0000
Epoch 75/100
 - 0s - loss: 0.0158 - categorical_accuracy: 1.0000 - val_loss: 0.0754 - val_categorical_accuracy: 1.0000
Epoch 76/100
 - 0s - loss: 0.0118 - categorical_accuracy: 1.0000 - val_loss: 0.0797 - val_categorical_accuracy: 1.0000
Epoch 77/100
 - 0s - loss: 0.0112 - categorical_accuracy: 1.0000 - val_loss: 0.0764 - val_categorical_accuracy: 1.0000
Epoch 78/100
 - 0s - loss: 0.0114 - categorical_

In [17]:
y_label=1*(y_predict>0.5)

In [18]:
y_label.reshape(-1,1).shape

(269, 1)

In [19]:
y_test.reshape(-1,1).shape

(269, 1)

In [24]:
#Test accuracy
1.0*sum((y_label.reshape(-1,1).flatten())==(y_test.reshape(-1,1).flatten()))/len(y_predict)

0.966542750929368

## Task 2. Classify all days of the week
Train a neural network against the origial categorical label. Use 5 layers of 40,15,5 and 7 (outputs, representing probabilities for a current input to correspond to each of the weekdays) neurons over the training sample, evaluating its performance over the test sample (use 'categorical_accurary'). Report the acheived accuracy (categorical) over the test sample.

First three layers use relu activation function, last one - sigmoid.
Use loss='binary_crossentropy', optimizer='adam', 200 epochs, batch_size=20

In [21]:
X_test=X[400:,:]; X_train=X[:400,:]; #split the data into training and test
y_test=yc[400:,:]; y_train=yc[:400,:]

In [22]:
model = Sequential()
model.add(Dense(units=40,activation='relu',input_dim=100))
model.add(Dense(units=15,activation='relu'))
model.add(Dense(units=5,activation='relu'))
model.add(Dense(units=7, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['categorical_accuracy'])
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=20,verbose=2)
y_predict=model.predict(X_test)

Train on 400 samples, validate on 269 samples
Epoch 1/200
 - 0s - loss: 0.6895 - categorical_accuracy: 0.1575 - val_loss: 0.6853 - val_categorical_accuracy: 0.1375
Epoch 2/200
 - 0s - loss: 0.6817 - categorical_accuracy: 0.2100 - val_loss: 0.6771 - val_categorical_accuracy: 0.2751
Epoch 3/200
 - 0s - loss: 0.6724 - categorical_accuracy: 0.2775 - val_loss: 0.6676 - val_categorical_accuracy: 0.2714
Epoch 4/200
 - 0s - loss: 0.6626 - categorical_accuracy: 0.2575 - val_loss: 0.6566 - val_categorical_accuracy: 0.2677
Epoch 5/200
 - 0s - loss: 0.6515 - categorical_accuracy: 0.2675 - val_loss: 0.6440 - val_categorical_accuracy: 0.2751
Epoch 6/200
 - 0s - loss: 0.6373 - categorical_accuracy: 0.2550 - val_loss: 0.6297 - val_categorical_accuracy: 0.2714
Epoch 7/200
 - 0s - loss: 0.6229 - categorical_accuracy: 0.2575 - val_loss: 0.6144 - val_categorical_accuracy: 0.2379
Epoch 8/200
 - 0s - loss: 0.6067 - categorical_accuracy: 0.2500 - val_loss: 0.6002 - val_categorical_accuracy: 0.1524
Epoch 9/20

Epoch 70/200
 - 0s - loss: 0.2119 - categorical_accuracy: 0.6750 - val_loss: 0.2529 - val_categorical_accuracy: 0.5948
Epoch 71/200
 - 0s - loss: 0.2117 - categorical_accuracy: 0.6725 - val_loss: 0.2559 - val_categorical_accuracy: 0.5688
Epoch 72/200
 - 0s - loss: 0.2098 - categorical_accuracy: 0.6725 - val_loss: 0.2636 - val_categorical_accuracy: 0.5688
Epoch 73/200
 - 0s - loss: 0.2089 - categorical_accuracy: 0.7000 - val_loss: 0.2518 - val_categorical_accuracy: 0.5985
Epoch 74/200
 - 0s - loss: 0.2081 - categorical_accuracy: 0.6750 - val_loss: 0.2576 - val_categorical_accuracy: 0.5985
Epoch 75/200
 - 0s - loss: 0.2076 - categorical_accuracy: 0.6700 - val_loss: 0.2478 - val_categorical_accuracy: 0.6729
Epoch 76/200
 - 0s - loss: 0.2070 - categorical_accuracy: 0.6950 - val_loss: 0.2522 - val_categorical_accuracy: 0.5799
Epoch 77/200
 - 0s - loss: 0.2068 - categorical_accuracy: 0.6850 - val_loss: 0.2512 - val_categorical_accuracy: 0.6283
Epoch 78/200
 - 0s - loss: 0.2058 - categorical_

Epoch 139/200
 - 0s - loss: 0.1673 - categorical_accuracy: 0.7675 - val_loss: 0.2448 - val_categorical_accuracy: 0.5874
Epoch 140/200
 - 0s - loss: 0.1686 - categorical_accuracy: 0.7450 - val_loss: 0.2331 - val_categorical_accuracy: 0.6357
Epoch 141/200
 - 0s - loss: 0.1654 - categorical_accuracy: 0.7825 - val_loss: 0.2291 - val_categorical_accuracy: 0.6729
Epoch 142/200
 - 0s - loss: 0.1655 - categorical_accuracy: 0.7775 - val_loss: 0.2371 - val_categorical_accuracy: 0.6171
Epoch 143/200
 - 0s - loss: 0.1655 - categorical_accuracy: 0.7575 - val_loss: 0.2409 - val_categorical_accuracy: 0.5911
Epoch 144/200
 - 0s - loss: 0.1651 - categorical_accuracy: 0.7775 - val_loss: 0.2399 - val_categorical_accuracy: 0.5911
Epoch 145/200
 - 0s - loss: 0.1638 - categorical_accuracy: 0.7775 - val_loss: 0.2291 - val_categorical_accuracy: 0.6617
Epoch 146/200
 - 0s - loss: 0.1640 - categorical_accuracy: 0.7800 - val_loss: 0.2571 - val_categorical_accuracy: 0.5688
Epoch 147/200
 - 0s - loss: 0.1664 - cat

In [36]:
y_predict=y_predict.argmax(axis=1)

In [49]:
y[400:]

array([1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1,
       2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2,
       3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3,
       4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4,
       5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5,
       6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6,
       0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0,
       1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1,
       2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2,
       3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3,
       4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4,
       5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5,
       6, 0, 1, 2, 3])

In [50]:
a=sum(y_predict==y[400:])
a

171

In [51]:
#Test accuracy
1.0*a/len(y_predict)

0.6356877323420075